In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from scipy import stats
from scipy.stats import norm, skew #for some statistics

In [ ]:
types = {'fare_amount': 'float32',
         'pickup_longitude': 'float32',
         'pickup_latitude': 'float32',
         'dropoff_longitude': 'float32',
         'dropoff_latitude': 'float32',
         'passenger_count': 'uint8'}
cols = ['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
def load_train_data(Nrows):
    return pd.read_csv('../input/train.csv', nrows=Nrows, dtype=types, usecols=cols, infer_datetime_format=True, parse_dates=["pickup_datetime"]) # total nrows = 55423855

In [ ]:
# Handling missing data and outliers; for feature engineering refer to the Pipe_line
def clean_missing_outliers(train_data):
    print('shape of before FE: ', train_data.shape)
    #handling missing data
    train_data.dropna(inplace=True)
    # removing outliers
    train_data.drop(train_data.loc[(train_data.fare_amount<=0) | (train_data.fare_amount>60)].index, inplace=True)
    train_data.drop(train_data.loc[(train_data.pickup_longitude<-74.03) | (train_data.pickup_longitude>-73.75)].index, inplace=True)
    train_data.drop(train_data.loc[(train_data.dropoff_longitude<-74.03) | (train_data.dropoff_longitude>-73.75)].index, inplace=True)
    train_data.drop(train_data.loc[(train_data.pickup_latitude<40.63) | (train_data.pickup_latitude>40.85)].index, inplace=True)
    train_data.drop(train_data.loc[(train_data.dropoff_latitude<40.63) | (train_data.dropoff_latitude>40.85)].index, inplace=True)
    train_data.drop(train_data.loc[(train_data.passenger_count>7)].index, inplace=True)

    print('shape of after FE: ', train_data.shape)
    return train_data

In [ ]:
# train test split
def tt_split(train_data):
    from sklearn.model_selection import train_test_split
    return train_test_split(train_data, test_size=0.2, random_state=23)

In [ ]:
# Get map_hour_dict & map_weekday_dict to be used in ModifyAttributes class
def get_mapping_dict(nrows=100, col='hour'):
    train_data = load_train_data(nrows)
    train_data = clean_missing_outliers(train_data)
    # ATTENTION: REMOVE STANDARDIZATION FROM THE PIPLELINE BEFORE RUNNING THIS FUNCTION
    train_data_tr, train_data_labels, pipeline, final_cols = Pipe_line(train_data)
    assembled_df = pd.DataFrame(np.c_[train_data_labels, train_data_tr], columns=['fare_amount'] + final_cols)
    assembled_df = assembled_df[assembled_df.distance_km >= 0.1]
    assembled_df.drop(assembled_df[(assembled_df.distance_km <= 0.5)].index, inplace=True)
    assembled_df['fare_per_km'] = assembled_df['fare_amount']/assembled_df['distance_km']
    summary_hour_duration = pd.DataFrame(assembled_df.astype(dtype='int32').groupby([col, 'year'])['fare_per_km'].mean())
    summary_hour_duration['unit']=1
    summary_hour_duration.reset_index(inplace=True)
    
    sns.tsplot(data=summary_hour_duration, time=col, unit = "unit", condition="year", value="fare_per_km")
    
    summary_hour_duration2 = summary_hour_duration.astype(dtype='int32').groupby([col])['fare_per_km'].mean()
    summary_hour_duration2.sort_values(ascending=True, inplace=True)
    print(summary_hour_duration2)
    output_dict = dict(zip(summary_hour_duration2.index, range(0, 25)))
    print(output_dict)

#get_mapping_dict(5000000, col='hour')

In [ ]:
# Transformation part of the Pipe_line given a block below
from sklearn.base import BaseEstimator, TransformerMixin
timeix, lat1ix, lon1ix, lat2ix, lon2ix = 0, 2, 1, 4, 3


class ModifyAttributes(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def distance(self, lat1, lon1, lat2, lon2):
        p = 0.017453292519943295 # Pi/180
        a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
        return 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
    #Feature engineering
        distance_km = [self.distance(X[_, lat1ix], X[_, lon1ix], X[_, lat2ix], X[_, lon2ix]) for _ in range(X.shape[0])]
        year = [_.year for _ in X[:, timeix]]
        month = [_.month for _ in X[:, timeix]]
        hour = [_.hour for _ in X[:, timeix]]
        map_hour_dict = {5: 0, 3: 1, 4: 2, 6: 3, 1: 4, 2: 5, 0: 6, 21: 7, 20: 8, 22: 9, 23: 10, 7: 11, 9: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 8: 21, 10: 22, 11: 23}
        mapped_hour = [map_hour_dict[_.hour] for _ in X[:, timeix]]
        map_weekday_dict = {6: 0, 5: 1, 0: 2, 1: 3, 2: 4, 3: 5, 4: 6}
        weekday = [_.weekday() for _ in X[:, timeix]]
        mapped_weekday = [map_weekday_dict[_.weekday()] for _ in X[:, timeix]]
    #     train_data['is_weekend'] = np.where(train_data.weekday<5, 0, 1)
    #     train_data['is_rush_hour'] = np.where((train_data.hour<8)  | (train_data.hour > 18), 0, 1)
        return np.c_[X[:, 1:], distance_km, year, month, hour, mapped_hour, weekday, mapped_weekday]

#train_data.drop(train_data.loc[(train_data.distance_km<=0.1)].index, inplace=True)


In [ ]:
# SKlearn's pipeline for dataset preparation
def Pipe_line(train_data):
    train_data_labels = train_data.fare_amount.copy()
    train_data = train_data.drop('fare_amount', axis=1)
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler

    pipeline = Pipeline([
        ('attribs_adder', ModifyAttributes()),
        ('std_scaler', StandardScaler())
    ])

    final_cols = list(train_data.columns[1:].values) + ['distance_km', 'year', 'month', 'hour', 'mapped_hour', 'weekday', 'mapped_weekday']

    train_data_tr = pipeline.fit_transform(train_data.values)
    train_data_tr = pd.DataFrame(train_data_tr, columns=final_cols)
    train_data_tr.head()
    return train_data_tr, train_data_labels, pipeline, final_cols

In [ ]:
# Get correlations - Pearson's r
def get_cors(nrows=100000):
    train_data = load_train_data(nrows)
    train_data = clean_missing_outliers(train_data)
    train_data, test_set = tt_split(train_data)
    train_data_tr, train_data_labels, pipeline, final_cols = Pipe_line(train_data)
    train_data_tr = pd.DataFrame(data=train_data_tr, columns=final_cols)
    
    # Plot correlations
    from scipy.stats import spearmanr
    import warnings
    warnings.filterwarnings("ignore")

    labels = []
    values = []
    for col in train_data_tr.columns:
        labels.append(col)
        values.append(spearmanr(train_data_tr[col].values, train_data_labels)[0])

    corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
    corr_df = corr_df.sort_values(by='corr_values')
    
    ind = np.arange(corr_df.shape[0])
    width = 0.9
    fig, ax = plt.subplots(figsize=(10,10))
    rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='red')
    ax.set_yticks(ind)
    ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
    ax.set_xlabel("Correlation coefficient")
    ax.set_title("Correlation coefficient of the variables")
    plt.grid()
    plt.show()

#get_cors(5000000)

# Model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
Kfolds = 5

def display_scores(scores):
    print('Scores:', scores)
    print('Mean RMSE:', scores.mean())
    print('STD of RMSE:', scores.std())

In [ ]:
# Linear Regression
def Lin_reg(train_data_tr, train_data_labels, cross_eval=False):
    from sklearn.linear_model import LinearRegression
    lin_reg = LinearRegression()
    lin_reg.fit(train_data_tr, train_data_labels)
    if cross_eval:
        lin_scores = cross_val_score(lin_reg, train_data_tr, train_data_labels, scoring='neg_mean_squared_error', cv=Kfolds)
        lin_rmse_scores = np.sqrt(-lin_scores)
        display_scores(lin_rmse_scores)
    return lin_reg

In [ ]:
# Decision Tree model
def Tree_reg(train_data_tr, train_data_labels, cross_eval=False):
    from sklearn.tree import DecisionTreeRegressor
    tree_reg = DecisionTreeRegressor()
    tree_reg.fit(train_data_tr, train_data_labels)
    if cross_eval:
        tree_scores = cross_val_score(tree_reg, train_data_tr, train_data_labels, scoring='neg_mean_squared_error', cv=Kfolds)
        tree_rmse_scores = np.sqrt(-tree_scores)
        display_scores(tree_rmse_scores)
    return tree_reg

In [ ]:
# XGBoost Regressor
def Xgb_reg(train_data_tr, train_data_labels, cross_eval=False):
    from xgboost import XGBRegressor
    xgb_reg = XGBRegressor(n_estimators=130, max_depth= 6, subsample=0.8, eta= 0.05, min_child_weight=10)
    xgb_reg.fit(train_data_tr, train_data_labels)
    if cross_eval:
        xgb_scores = cross_val_score(xgb_reg, train_data_tr, train_data_labels, scoring='neg_mean_squared_error', cv=Kfolds)
        xgb_rmse_scores = np.sqrt(-xgb_scores)
        display_scores(xgb_rmse_scores)
    return xgb_reg


In [ ]:
# SGD Regressor
def Sgd_reg(train_data_tr, train_data_labels, cross_eval=False):
    from sklearn.linear_model import SGDRegressor
    sgd_reg = SGDRegressor()
    sgd_reg.fit(train_data_tr, train_data_labels)
    if cross_eval:
        sgd_scores = cross_val_score(sgd_reg, train_data_tr, train_data_labels, scoring='neg_mean_squared_error', cv=Kfolds)
        sgd_rmse_scores = np.sqrt(-sgd_scores)
        display_scores(sgd_rmse_scores)
    return sgd_reg

In [ ]:
# Random Forest model
def Forest_reg(train_data_tr, train_data_labels, cross_eval=False):
    from sklearn.ensemble import RandomForestRegressor
    forest_reg = RandomForestRegressor(n_estimators=130, max_features=5)
    forest_reg.fit(train_data_tr, train_data_labels)
    if cross_eval:
        forest_scores = cross_val_score(forest_reg, train_data_tr, train_data_labels, scoring='neg_mean_squared_error', cv=Kfolds)
        forest_rmse_scores = np.sqrt(-forest_scores)
        display_scores(forest_rmse_scores)
    return forest_reg

In [ ]:
# Fine-tune random forest
# from sklearn.model_selection import GridSearchCV
# param_grid = [
#     {'n_estimators': [3, 10, 30, 100], 'max_features': [2, 4, 6, 8]}
# ]
# grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(train_data_tr, train_data_labels)
# grid_search.best_params_

In [ ]:
# cvres = grid_search.cv_results_
# for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
#     print(np.sqrt(-mean_score), params)

In [ ]:
# Fine - tune eXtreme Gradient Boost
def fine_tune_xgb(nrows):
    train_data = load_train_data(nrows)
    train_data = clean_missing_outliers(train_data)
    train_data, test_set = tt_split(train_data)
    train_data_tr, train_data_labels, pipeline, final_cols = Pipe_line(train_data)
    test_data_tr, test_data_labels, pipeline, final_cols = Pipe_line(test_set)
    
    import xgboost as xgb
    FOREVER_COMPUTING_FLAG = False
    xgb_pars = []
    for MCW in [10, 20, 50, 75, 100]:
        for ETA in [0.05, 0.1, 0.15]:
            for CS in [0.3, 0.4, 0.5]:
                for MD in [6, 8, 10, 12, 15]:
                    for SS in [0.5, 0.6, 0.7, 0.8, 0.9]:
                        for LAMBDA in [0.5, 1., 1.5,  2., 3.]:
                            xgb_pars.append({'min_child_weight': MCW, 'eta': ETA, 
                                             'colsample_bytree': CS, 'max_depth': MD,
                                             'subsample': SS, 'lambda': LAMBDA, 
                                             'nthread': -1, 'booster' : 'gbtree', 'eval_metric': 'rmse',
                                             'silent': 1, 'objective': 'reg:linear'})

    dtrain = xgb.DMatrix(train_data_tr, label=train_data_labels)
    dvalid = xgb.DMatrix(test_data_tr, label=test_data_labels)
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    for i in range(100):
        xgb_par = np.random.choice(xgb_pars, 1)[0]
        print(xgb_par)
        model = xgb.train(xgb_par, dtrain, 2000, watchlist, early_stopping_rounds=50,
                          maximize=False, verbose_eval=100)
        print('Modeling RMSE %.5f' % model.best_score)

#fine_tune_xgb(nrows=1000)

In [ ]:
# best_model = grid_search.best_estimator_

# Evaluate on the Test Set

In [ ]:
def evaluate_model(model, pipeline, test_set, final_cols):
    X_test = test_set.drop(['fare_amount'], axis=1)
    y_test = test_set.fare_amount.values
    X_test_prepared = pipeline.transform(X_test.values)
    X_test_prepared = pd.DataFrame(X_test_prepared, columns=final_cols)
    final_predictions = model.predict(X_test_prepared)
    final_mse = mean_squared_error(y_test, final_predictions)
    final_rmse = np.sqrt(final_mse)
    plt.scatter(y_test, final_predictions, alpha=0.3)
    plt.xlabel('measured')
    plt.ylabel('predicted')
    plt.show()
    fig = plt.figure()
    res = stats.probplot((y_test - final_predictions), plot=plt)
    plt.show()
    print('RMSE on evaluation set: ', final_rmse)

# Launch! (predict test.csv)

In [ ]:
def write_test_file(model, nrows, model_name, pipeline, final_cols, train_data_labels, brute_force=False):
    test_csv = pd.read_csv('../input/test.csv', dtype=types, infer_datetime_format=True, parse_dates=["pickup_datetime"])
    X_test_csv_keys = test_csv.key.copy()
    X_test_csv = test_csv.drop('key', axis=1)
    X_test_csv_prepared = pipeline.transform(X_test_csv.values)
    X_test_csv_prepared = pd.DataFrame(X_test_csv_prepared, columns=final_cols)
    final_predictions_csv = model.predict(X_test_csv_prepared)
    print(final_predictions_csv)
    # Apply brute force rules...
    def brute_force_results(inp_data, train_data_labels):
        q1 = train_data_labels.quantile(0.0042)
        q2 = train_data_labels.quantile(0.99)
        
        dataframe = pd.DataFrame(inp_data)
        dataframe[0] = dataframe[0].apply(lambda x: x if x > q1 else x*1.1)
        dataframe[0] = dataframe[0].apply(lambda x: x if x < q2 else x*0.77)
        return dataframe[0].values
    
    brt_str =''
    if brute_force:
        final_predictions_csv = brute_force_results(final_predictions_csv, train_data_labels)
        brt_str = '_w_bf'
    
    submission = pd.DataFrame(data={'key':X_test_csv_keys, 'fare_amount': final_predictions_csv})
    print(submission.head())
    filename = 'submission_w_' + str(nrows) + '-training_w_' + str(model_name.__name__) + brt_str + '.csv'
    submission.to_csv(filename, index=None)
    print(filename, ' file is written.')

In [ ]:
import time
def run_entire_process(model=Lin_reg, cross_eval=False, test_eval=False, launch=False, nrows=1000, brute_force=False):
    print('running ', str(model.__name__), ' on ', str(nrows), ' training rows with cross_eval=', str(cross_eval),
         'test_eval=', str(test_eval), ' launch=', str(launch))
    start_time = time.time()
    train_data = load_train_data(nrows)
    train_data = clean_missing_outliers(train_data)
    train_data, test_set = tt_split(train_data)
    train_data_tr, train_data_labels, pipeline, final_cols = Pipe_line(train_data)
    modl = model(train_data_tr, train_data_labels, cross_eval)
    if test_eval:
        evaluate_model(modl, pipeline, test_set, final_cols)
    if launch:
        write_test_file(modl, nrows, model, pipeline, final_cols, train_data_labels, brute_force)
    print('run time = ', time.time() - start_time, ' secs')
    
Models = [Lin_reg, Tree_reg, Xgb_reg, Sgd_reg, Forest_reg]
# for _ in Models:
#     run_entire_process(model=_, cross_eval=True, test_eval=True, launch=False, nrows=1000000)
run_entire_process(model=Models[2], cross_eval=True, test_eval=True, launch=True, nrows=1000000, brute_force=False)